In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd
import re

In [ ]:
sic4_desc = spark.sql('select csic4 as sic4, sic4_description from workarea.sic4_desc')

wblinkage = spark.sql('select duns_nbr as wb_duns, phy_ctry_code as wb_country_code, phy_ctry_nme as wb_country_name, cast(ltrim(rtrim(us_1987_sics_1)) as int) as wb_sic4, load_year as wb_load_year, load_month as wb_load_month from workarea.wblinkage where load_year in (2015, 2016, 2017)')

wbusunlinked = spark.sql('select duns_nbr as wb_duns, phy_ctry_code as wb_country_code, phy_ctry_nme as wb_country_name, cast(ltrim(rtrim(sic_base_1)) as int) as wb_sic4, load_year as wb_load_year, load_month as wb_load_month from workarea.wbusunlinked where load_year in (2015, 2016, 2017)')

wbglobalunlinked = spark.sql('select duns_nbr as wb_duns, phy_ctry_code as wb_country_code, phy_ctry_nme as wb_country_name, cast(ltrim(rtrim(us_1987_sics_1)) as int) as wb_sic4, load_year as wb_load_year, load_month as wb_load_month from workarea.wbglobalunlinked where load_year in (2015, 2016, 2017)')

wb = wblinkage.union(wbusunlinked).union(wbglobalunlinked)

trade_2016 = spark.sql('select supp_duns_nbr as supplier_duns, rel_duns_nbr as buyer_duns, load_year, load_month from workarea.gt_purc_pstg_data_type3 where load_year in (2014, 2015, 2016)')

trade_2016 = trade_2016.join(wb, (col('supplier_duns') == col('wb_duns')) & (col('load_year') == col('wb_load_year')) & (col('load_month') == col('wb_load_month')), how = 'left')
trade_2016 = trade_2016.withColumnRenamed('wb_country_code', 'supplier_country_code')
trade_2016 = trade_2016.withColumnRenamed('wb_country_name', 'supplier_country_name')
trade_2016 = trade_2016.withColumnRenamed('wb_sic4', 'supplier_sic4')
trade_2016 = trade_2016.drop(*['wb_duns', 'wb_load_year', 'wb_load_month'])
trade_2016 = trade_2016.join(wb, (col('buyer_duns') == col('wb_duns')) & (col('load_year') == col('wb_load_year')) & (col('load_month') == col('wb_load_month')), how = 'left')
trade_2016 = trade_2016.withColumnRenamed('wb_country_code', 'buyer_country_code')
trade_2016 = trade_2016.withColumnRenamed('wb_country_name', 'buyer_country_name')
trade_2016 = trade_2016.withColumnRenamed('wb_sic4', 'buyer_sic4')
trade_2016 = trade_2016.drop(*['wb_duns', 'wb_load_year', 'wb_load_month'])

trade_2016_timeperiod = trade_2016.groupBy(['load_year', 'load_month']).count().orderBy(['load_year', 'load_month'])
trade_2016_timeperiod.registerTempTable('tb')
trade_2016_timeperiod2 = spark.sql("""
select load_year, load_month, row_number() over (order by load_year desc, load_month desc) as id
from tb 
""")
trade_2016 = trade_2016.join(trade_2016_timeperiod2, on = ['load_year', 'load_month'], how = 'left')
trade_2016 = trade_2016.where('id >= 4 AND id <= 28')

trade_2016 = trade_2016.withColumn('supplier_append_year', lit(2016))
trade_2016 = trade_2016.withColumn('supplier_append_month', lit(9))

trade_2017 = spark.sql('select supp_duns_nbr as supplier_duns, rel_duns_nbr as buyer_duns, load_year, load_month from workarea.gt_purc_pstg_data_type3 where load_year in (2015, 2016, 2017)')

trade_2017 = trade_2017.join(wb, (col('supplier_duns') == col('wb_duns')) & (col('load_year') == col('wb_load_year')) & (col('load_month') == col('wb_load_month')), how = 'left')
trade_2017 = trade_2017.withColumnRenamed('wb_country_code', 'supplier_country_code')
trade_2017 = trade_2017.withColumnRenamed('wb_country_name', 'supplier_country_name')
trade_2017 = trade_2017.withColumnRenamed('wb_sic4', 'supplier_sic4')
trade_2017 = trade_2017.drop(*['wb_duns', 'wb_load_year', 'wb_load_month'])
trade_2017 = trade_2017.join(wb, (col('buyer_duns') == col('wb_duns')) & (col('load_year') == col('wb_load_year')) & (col('load_month') == col('wb_load_month')), how = 'left')
trade_2017 = trade_2017.withColumnRenamed('wb_country_code', 'buyer_country_code')
trade_2017 = trade_2017.withColumnRenamed('wb_country_name', 'buyer_country_name')
trade_2017 = trade_2017.withColumnRenamed('wb_sic4', 'buyer_sic4')
trade_2017 = trade_2017.drop(*['wb_duns', 'wb_load_year', 'wb_load_month'])

trade_2017_timeperiod = trade_2017.groupBy(['load_year', 'load_month']).count().orderBy(['load_year', 'load_month'])
trade_2017_timeperiod.registerTempTable('tb')
trade_2017_timeperiod2 = spark.sql("""
select load_year, load_month, row_number() over (order by load_year desc, load_month desc) as id
from tb 
""")
trade_2017 = trade_2017.join(trade_2017_timeperiod2, on = ['load_year', 'load_month'], how = 'left')
trade_2017 = trade_2017.where('id >= 4 AND id <= 28')

trade_2017 = trade_2017.withColumn('supplier_append_year', lit(2017))
trade_2017 = trade_2017.withColumn('supplier_append_month', lit(9))

trade = trade_2016.union(trade_2017)

trade_final = trade[['supplier_duns', 'buyer_duns', 'supplier_country_code', 'supplier_country_name', 'supplier_sic4', 'buyer_country_code', 'buyer_country_name', 'buyer_sic4', 'supplier_append_year', 'supplier_append_month']]

trade_final = trade_final.withColumn('trade_match_ind', lit(1))

train = spark.sql('select * from workarea.us_export_propensity_analytic_dataset_train')
train2 = train.join(trade_final, ((col('duns') == col('supplier_duns')) & (col('append_year') == col('supplier_append_year')) & (col('append_month') == col('supplier_append_month'))), how = 'left')
train2.write.mode('overwrite').saveAsTable('workarea.us_export_propensity_analytic_dataset_train_trade_data')

val = spark.sql('select * from workarea.us_export_propensity_analytic_dataset_val')
val2 = val.join(trade_final, ((col('duns') == col('supplier_duns')) & (col('append_year') == col('supplier_append_year')) & (col('append_month') == col('supplier_append_month'))), how = 'left')
val2.write.mode('overwrite').saveAsTable('workarea.us_export_propensity_analytic_dataset_val_trade_data')

test = spark.sql('select * from workarea.us_export_propensity_analytic_dataset_test')
test2 = test.join(trade_final, ((col('duns') == col('supplier_duns')) & (col('append_year') == col('supplier_append_year')) & (col('append_month') == col('supplier_append_month'))), how = 'left')
test2.write.mode('overwrite').saveAsTable('workarea.us_export_propensity_analytic_dataset_test_trade_data')

trade_data_train = spark.sql('select * from workarea.us_export_propensity_analytic_dataset_train_trade_data')
trade_data_train = trade_data_train.withColumn('non_us_trade', lit(None))
trade_data_train = trade_data_train.withColumn('non_us_trade', when(col('buyer_country_code') != 805, 1).otherwise(col('non_us_trade')))
trade_data_train = trade_data_train.withColumn('non_us_trade', when(col('buyer_country_code') == 805, 0).otherwise(col('non_us_trade')))
trade_data_train_grp = trade_data_train.groupBy(['duns', 'append_year', 'append_month', 'export', 'non_us_trade']).count().orderBy(['duns', 'append_year', 'append_month', 'export', 'non_us_trade'])
trade_data_train_grp2 = trade_data_train_grp.groupBy(['duns', 'append_year', 'append_month', 'export']).agg(max('non_us_trade').alias('non_us_trade')).orderBy(['duns', 'append_year', 'append_month', 'export'])

trade_data_val = spark.sql('select * from workarea.us_export_propensity_analytic_dataset_val_trade_data')
trade_data_val = trade_data_val.withColumn('non_us_trade', lit(None))
trade_data_val = trade_data_val.withColumn('non_us_trade', when(col('buyer_country_code') != 805, 1).otherwise(col('non_us_trade')))
trade_data_val = trade_data_val.withColumn('non_us_trade', when(col('buyer_country_code') == 805, 0).otherwise(col('non_us_trade')))
trade_data_val_grp = trade_data_val.groupBy(['duns', 'append_year', 'append_month', 'export', 'non_us_trade']).count().orderBy(['duns', 'append_year', 'append_month', 'export', 'non_us_trade'])
trade_data_val_grp2 = trade_data_val_grp.groupBy(['duns', 'append_year', 'append_month', 'export']).agg(max('non_us_trade').alias('non_us_trade')).orderBy(['duns', 'append_year', 'append_month', 'export'])

trade_data_test = spark.sql('select * from workarea.us_export_propensity_analytic_dataset_test_trade_data')
trade_data_test = trade_data_test.withColumn('non_us_trade', lit(None))
trade_data_test = trade_data_test.withColumn('non_us_trade', when(col('buyer_country_code') != 805, 1).otherwise(col('non_us_trade')))
trade_data_test = trade_data_test.withColumn('non_us_trade', when(col('buyer_country_code') == 805, 0).otherwise(col('non_us_trade')))
trade_data_test_grp = trade_data_test.groupBy(['duns', 'append_year', 'append_month', 'export', 'non_us_trade']).count().orderBy(['duns', 'append_year', 'append_month', 'export', 'non_us_trade'])
trade_data_test_grp2 = trade_data_test_grp.groupBy(['duns', 'append_year', 'append_month', 'export']).agg(max('non_us_trade').alias('non_us_trade')).orderBy(['duns', 'append_year', 'append_month', 'export'])

shipping = spark.sql('select shipper_duns, shipper_country_nme, shipper_sic4, consignee_duns, consignee_country_nme, consignee_sic4, est_arrival_tme from workarea.jzhang_shipments_all_cm_final6_temp1')

shipping = shipping.withColumn('est_arrival_year', year(col('est_arrival_tme')))
shipping = shipping.withColumn('est_arrival_month', month(col('est_arrival_tme')))

shipping_2016 = shipping.where('est_arrival_year in (2015, 2016, 2017)')
shipping_2017 = shipping.where('est_arrival_year in (2016, 2017, 2018)')

shipping_2016 = shipping_2016.withColumn('shipping_append_year', lit(2016))
shipping_2016 = shipping_2016.withColumn('shipping_append_month', lit(9))
shipping_2017 = shipping_2017.withColumn('shipping_append_year', lit(2017))
shipping_2017 = shipping_2017.withColumn('shipping_append_month', lit(9))

shipping_final = shipping_2016.union(shipping_2017)

shipping_final = shipping_final.withColumn('shipping_match_ind', lit(1))

train = spark.sql('select * from workarea.us_export_propensity_analytic_dataset_train')
train2 = train.join(shipping_final, (col('duns') == col('shipper_duns')) & (col('append_year') == col('shipping_append_year')) & (col('append_month') == col('shipping_append_month')), how = 'left')

train3 = train2.join(sic4_desc, col('shipper_sic4') == col('sic4'), how = 'left')
train3 = train3.drop('sic4')
train3 = train3.withColumnRenamed('sic4_description', 'shipper_sic4_description')
train4 = train3.join(sic4_desc, col('consignee_sic4') == col('sic4'), how = 'left')
train4 = train4.drop('sic4')
train4 = train4.withColumnRenamed('sic4_description', 'consignee_sic4_description')

train4 = train4.withColumn('non_us_shipment', lit(None))
train4 = train4.withColumn('non_us_shipment', when(~col('consignee_country_nme').isin(["US", "UNITED STATES"]), 1).otherwise(col('non_us_shipment')))
train4 = train4.withColumn('non_us_shipment', when(col('consignee_country_nme').isin(["US", "UNITED STATES"]), 0).otherwise(col('non_us_shipment')))

train4.write.mode('overwrite').saveAsTable('workarea.us_export_propensity_analytic_dataset_train_shipping_data')

val = spark.sql('select * from workarea.us_export_propensity_analytic_dataset_val')
val2 = val.join(shipping_final, (col('duns') == col('shipper_duns')) & (col('append_year') == col('shipping_append_year')) & (col('append_month') == col('shipping_append_month')), how = 'left')

val3 = val2.join(sic4_desc, col('shipper_sic4') == col('sic4'), how = 'left')
val3 = val3.drop('sic4')
val3 = val3.withColumnRenamed('sic4_description', 'shipper_sic4_description')
val4 = val3.join(sic4_desc, col('consignee_sic4') == col('sic4'), how = 'left')
val4 = val4.drop('sic4')
val4 = val4.withColumnRenamed('sic4_description', 'consignee_sic4_description')

val4 = val4.withColumn('non_us_shipment', lit(None))
val4 = val4.withColumn('non_us_shipment', when(~col('consignee_country_nme').isin(["US", "UNITED STATES"]), 1).otherwise(col('non_us_shipment')))
val4 = val4.withColumn('non_us_shipment', when(col('consignee_country_nme').isin(["US", "UNITED STATES"]), 0).otherwise(col('non_us_shipment')))

val4.write.mode('overwrite').saveAsTable('workarea.us_export_propensity_analytic_dataset_val_shipping_data')

test = spark.sql('select * from workarea.us_export_propensity_analytic_dataset_test')
test2 = test.join(shipping_final, (col('duns') == col('shipper_duns')) & (col('append_year') == col('shipping_append_year')) & (col('append_month') == col('shipping_append_month')), how = 'left')

test3 = test2.join(sic4_desc, col('shipper_sic4') == col('sic4'), how = 'left')
test3 = test3.drop('sic4')
test3 = test3.withColumnRenamed('sic4_description', 'shipper_sic4_description')
test4 = test3.join(sic4_desc, col('consignee_sic4') == col('sic4'), how = 'left')
test4 = test4.drop('sic4')
test4 = test4.withColumnRenamed('sic4_description', 'consignee_sic4_description')

test4 = test4.withColumn('non_us_shipment', lit(None))
test4 = test4.withColumn('non_us_shipment', when(~col('consignee_country_nme').isin(["US", "UNITED STATES"]), 1).otherwise(col('non_us_shipment')))
test4 = test4.withColumn('non_us_shipment', when(col('consignee_country_nme').isin(["US", "UNITED STATES"]), 0).otherwise(col('non_us_shipment')))

test4.write.mode('overwrite').saveAsTable('workarea.us_export_propensity_analytic_dataset_test_shipping_data')

shipping_data_train = spark.sql('select * from workarea.us_export_propensity_analytic_dataset_train_shipping_data')
shipping_data_train_grp = shipping_data_train.groupBy(['duns', 'append_year', 'append_month', 'export', 'non_us_shipment']).count().orderBy(['duns', 'append_year', 'append_month', 'export', 'non_us_shipment'])
shipping_data_train_grp2 = shipping_data_train_grp.groupBy(['duns', 'append_year', 'append_month', 'export']).agg(max('non_us_shipment').alias('non_us_shipment')).orderBy(['duns', 'append_year', 'append_month', 'export'])

shipping_data_val = spark.sql('select * from workarea.us_export_propensity_analytic_dataset_val_shipping_data')
shipping_data_val_grp = shipping_data_val.groupBy(['duns', 'append_year', 'append_month', 'export', 'non_us_shipment']).count().orderBy(['duns', 'append_year', 'append_month', 'export', 'non_us_shipment'])
shipping_data_val_grp2 = shipping_data_val_grp.groupBy(['duns', 'append_year', 'append_month', 'export']).agg(max('non_us_shipment').alias('non_us_shipment')).orderBy(['duns', 'append_year', 'append_month', 'export'])

shipping_data_test = spark.sql('select * from workarea.us_export_propensity_analytic_dataset_test_shipping_data')
shipping_data_test_grp = shipping_data_test.groupBy(['duns', 'append_year', 'append_month', 'export', 'non_us_shipment']).count().orderBy(['duns', 'append_year', 'append_month', 'export', 'non_us_shipment'])
shipping_data_test_grp2 = shipping_data_test_grp.groupBy(['duns', 'append_year', 'append_month', 'export']).agg(max('non_us_shipment').alias('non_us_shipment')).orderBy(['duns', 'append_year', 'append_month', 'export'])

shipping_data_train_grp2 = shipping_data_train_grp2.withColumn('sample_type', lit('train'))
shipping_data_val_train_grp2 = shipping_data_val_grp2.withColumn('sample_type', lit('val'))
shipping_data_test_train_grp2 = shipping_data_test_grp2.withColumn('sample_type', lit('test'))
shipping_data_final = shipping_data_train_grp2.union(shipping_data_val_train_grp2).union(shipping_data_test_train_grp2)
shipping_data_final = shipping_data_final.withColumnRenamed('non_us_shipment', 'foreign_shipment_ind')
shipping_data_final.write.mode('overwrite').saveAsTable('workarea.us_export_propensity_analytic_dataset_foreign_shipment_derived_attribute')

shipping_data_train_grp2.count(), shipping_data_val_grp2.count(), shipping_data_test_grp2.count()

trade_data_train_grp2 = trade_data_train_grp2.withColumn('sample_type', lit('train'))
trade_data_val_train_grp2 = trade_data_val_grp2.withColumn('sample_type', lit('val'))
trade_data_test_train_grp2 = trade_data_test_grp2.withColumn('sample_type', lit('test'))
trade_data_final = trade_data_train_grp2.union(trade_data_val_train_grp2).union(trade_data_test_train_grp2)
trade_data_final = trade_data_final.withColumnRenamed('non_us_trade', 'foreign_trade_ind')
trade_data_final.write.mode('overwrite').saveAsTable('workarea.us_export_propensity_analytic_dataset_foreign_trade_derived_attribute')